In [8]:
import os
import shutil
from pathlib import Path
import json
from PIL import Image
from tqdm import tqdm

# def create_training_list(dataset_path, output_path, split='train'):
#     """
#     Create training list file in PaddleOCR format
#     Format: image_path\tlabel
#     """
#     data_dir = Path(dataset_path) / split
#     output_file = Path(output_path) / f'{split}_list.txt'
    
#     with open(output_file, 'w', encoding='utf-8') as f:
#         for img_path in data_dir.glob('*.*'):
#             if img_path.suffix.lower() in ['.png', '.jpg', '.jpeg']:
#                 # Get label from filename
#                 label = img_path.stem
#                 # Write in PaddleOCR format
#                 f.write(f'{str(img_path)}\t{label}\n')
    
#     print(f"Created {split} list at: {output_file}")
def create_training_list(dataset_path, output_path, split='train'):
    """
    Create training list file in PaddleOCR format
    Format: image_path\tlabel
    """
    data_dir = Path(dataset_path) / split
    output_file = Path(output_path) / f'{split}_list.txt'
    
    with open(output_file, 'w', encoding='utf-8') as f:
        for img_path in data_dir.glob('*.*'):
            if img_path.suffix.lower() in ['.png', '.jpg', '.jpeg']:
                # Get label from filename
                label = img_path.stem
                # Write in PaddleOCR format
                f.write(f'{str(img_path)}\t{label}\n')
    
    print(f"Created {split} list at: {output_file}")

def create_dict_file(dataset_path, output_path):
    """
    Create dictionary file containing all characters in the dataset
    """
    chars = set()
    
    # Collect all unique characters from all splits
    for split in ['train', 'val', 'test']:
        data_dir = Path(dataset_path) / split
        for img_path in data_dir.glob('*.*'):
            if img_path.suffix.lower() in ['.png', '.jpg', '.jpeg']:
                chars.update(img_path.stem)
    
    # Sort and write to file
    chars = sorted(list(chars))
    dict_file = Path(output_path) / 'dict.txt'
    
    with open(dict_file, 'w', encoding='utf-8') as f:
        for char in chars:
            f.write(f'{char}\n')
    
    print(f"Created dictionary file at: {dict_file}")
    return len(chars)

def create_config(output_path, num_classes):
    """
    Create PaddleOCR training configuration
    """
    config = {
        "Architecture": {
            "model_type": "rec",
            "algorithm": "CRNN",
            "Transform": None,
            "Backbone": {
                "name": "ResNet",
                "layers": 34
            },
            "Neck": {
                "name": "SequenceEncoder",
                "encoder_type": "rnn",
                "hidden_size": 256
            },
            "Head": {
                "name": "CTCHead",
                "fc_decay": 0.0004
            }
        },
        "Loss": {
            "name": "CTCLoss"
        },
        "Optimizer": {
            "name": "Adam",
            "beta1": 0.9,
            "beta2": 0.999,
            "lr": {
                "name": "Cosine",
                "learning_rate": 0.001,
                "warmup_epoch": 5
            }
        },
        "Train": {
            "dataset": {
                "name": "SimpleDataSet",
                "data_dir": "dataset/",
                "label_file_list": ["train_list.txt"],
                "transforms": [
                    {"DecodeImage": {"img_mode": "RGB", "channel_first": False}},
                    {"RecResizeImg": {"image_shape": [3, 32, 100]}},
                    {"KeepKeys": {"keep_keys": ["image", "label"]}}
                ]
            },
            "loader": {
                "batch_size_per_card": 32,
                "drop_last": True,
                "num_workers": 4,
                "shuffle": True,
            }
        },
        "Eval": {
            "dataset": {
                "name": "SimpleDataSet",
                "data_dir": "dataset/",
                "label_file_list": ["val_list.txt"],
                "transforms": [
                    {"DecodeImage": {"img_mode": "RGB", "channel_first": False}},
                    {"RecResizeImg": {"image_shape": [3, 32, 100]}},
                    {"KeepKeys": {"keep_keys": ["image", "label"]}}
                ]
            },
            "loader": {
                "batch_size_per_card": 32,
                "drop_last": False,
                "num_workers": 4,
                "shuffle": False,
            }
        },
        "Global": {
            "epoch_num": 100,
            "save_model_dir": "./output/rec_crnn",
            "save_epoch_step": 5,
            "eval_batch_step": [0, 2000],
            "cal_metric_during_train": True,
            "pretrained_model": "./pretrain_models/CRNN/best_accuracy",
            "checkpoints": None,
            "save_inference_dir": None,
            "use_visualdl": True,
            "class_num": num_classes + 1,  # Add 1 for blank token
            "character_dict_path": "dict.txt",
            "character_type": "ch",
            "use_space_char": False
        }
    }
    
    config_file = Path(output_path) / 'config.yml'
    with open(config_file, 'w') as f:
        json.dump(config, f, indent=2)
    
    print(f"Created configuration file at: {config_file}")

def prepare_training():
    """
    Prepare all necessary files for PaddleOCR training
    """
    # Create output directory
    output_path = Path('paddleocr_training')
    output_path.mkdir(exist_ok=True)
    
    # Create training lists
    for split in ['train', 'val', 'test']:
        create_training_list('dataset', output_path, split)
    
    # Create dictionary
    num_classes = create_dict_file('dataset', output_path)
    
    # Create configuration
    create_config(output_path, num_classes)
    
    print("\nTraining preparation completed!")
    print("\nTo start training, follow these steps:")
    print("1. Install PaddlePaddle and PaddleOCR:")
    print("   pip install paddlepaddle-gpu paddleocr")
    print("\n2. Run training:")
    print("   python PaddleOCR/tools/train.py -c paddleocr_training/config.yml")

# if __name__ == "__main__":
#     prepare_training()

In [9]:
prepare_training()

Created train list at: paddleocr_training\train_list.txt
Created val list at: paddleocr_training\val_list.txt
Created test list at: paddleocr_training\test_list.txt
Created dictionary file at: paddleocr_training\dict.txt
Created configuration file at: paddleocr_training\config.yml

Training preparation completed!

To start training, follow these steps:
1. Install PaddlePaddle and PaddleOCR:
   pip install paddlepaddle-gpu paddleocr

2. Run training:
   python PaddleOCR/tools/train.py -c paddleocr_training/config.yml


In [ ]:
# def create_training_list(dataset_path, output_path, split='train'):
#     """
#     Create training list file in PaddleOCR format
#     Format: image_path\tlabel
#     """
#     data_dir = Path(dataset_path) / split
#     output_file = Path(output_path) / f'{split}_list.txt'
    
#     with open(output_file, 'w', encoding='utf-8') as f:
#         for img_path in data_dir.glob('*.*'):
#             if img_path.suffix.lower() in ['.png', '.jpg', '.jpeg']:
#                 # Get label from filename
#                 label = img_path.stem
#                 label = re.sub(r'_.*', '', label)
#                 # Write in PaddleOCR format
#                 rel_path = img_path.relative_to(Path(dataset_path))
#                 f.write(f'{rel_path}\t{label}\n')
    
#     print(f"Created {split} list at: {output_file}")

# def create_dict_file(dataset_path, output_path):
#     """
#     Create dictionary file containing all characters in the dataset
#     """
#     chars = set()
    
#     # Collect all unique characters from all splits
#     for split in ['train', 'val', 'test']:
#         data_dir = Path(dataset_path) / split
#         for img_path in data_dir.glob('*.*'):
#             if img_path.suffix.lower() in ['.png', '.jpg', '.jpeg']:
#                 chars.update(img_path.stem)
    
#     # Sort and write to file
#     chars = sorted(list(chars))
#     dict_file = Path(output_path) / 'dict.txt'
    
#     with open(dict_file, 'w', encoding='utf-8') as f:
#         for char in chars:
#             f.write(f'{char}\n')
    
#     print(f"Created dictionary file at: {dict_file}")
#     return len(chars)

# def create_config(output_path, num_classes):
#     """
#     Create PaddleOCR training configuration
#     """
#     config = {
#         "Architecture": {
#             "model_type": "rec",
#             "algorithm": "CRNN",
#             "Transform": None,
#             "Backbone": {
#                 "name": "ResNet",
#                 "layers": 34
#             },
#             "Neck": {
#                 "name": "SequenceEncoder",
#                 "encoder_type": "rnn",
#                 "hidden_size": 256
#             },
#             "Head": {
#                 "name": "CTCHead",
#                 "fc_decay": 0.0004
#             }
#         },
#         "Loss": {
#             "name": "CTCLoss"
#         },
#         "Optimizer": {
#             "name": "Adam",
#             "beta1": 0.9,
#             "beta2": 0.999,
#             "lr": {
#                 "name": "Cosine",
#                 "learning_rate": 0.001,
#                 "warmup_epoch": 5
#             }
#         },
#         "Train": {
#             "dataset": {
#                 "name": "SimpleDataSet",
#                 "data_dir": "dataset/",
#                 "label_file_list": ["paddleocr_training/train_list.txt"],
#                 "transforms": [
#                     {"DecodeImage": {"img_mode": "RGB", "channel_first": False}},
#                     {"RecResizeImg": {"image_shape": [3, 32, 100]}},
#                     {"KeepKeys": {"keep_keys": ["image", "label"]}}
#                 ]
#             },
#             "loader": {
#                 "batch_size_per_card": 32,
#                 "drop_last": True,
#                 "num_workers": 4,
#                 "shuffle": True,
#             }
#         },
#         "Eval": {
#             "dataset": {
#                 "name": "SimpleDataSet",
#                 "data_dir": "dataset/",
#                 "label_file_list": ["paddleocr_training/val_list.txt"],
#                 "transforms": [
#                     {"DecodeImage": {"img_mode": "RGB", "channel_first": False}},
#                     {"RecResizeImg": {"image_shape": [3, 32, 100]}},
#                     {"KeepKeys": {"keep_keys": ["image", "label"]}}
#                 ]
#             },
#             "loader": {
#                 "batch_size_per_card": 32,
#                 "drop_last": False,
#                 "num_workers": 4,
#                 "shuffle": False,
#             }
#         },
#         "Global": {
#             "epoch_num": 100,
#             "save_model_dir": "./output/rec_crnn",
#             "save_epoch_step": 5,
#             "eval_batch_step": [0, 2000],
#             "cal_metric_during_train": True,
#             "pretrained_model": "./pretrain_models/en_PP-OCRv4_rec_train/best_accuracy",
#             "checkpoints": None,
#             "save_inference_dir": None,
#             "use_visualdl": True,
#             "class_num": num_classes + 1,  # Add 1 for blank token
#             "character_dict_path": "paddleocr_training/dict.txt",
#             "character_type": "ch",
#             "use_space_char": False,
#             "log_smooth_window": 20,
#             "print_batch_step": 10
#         },
#         "PostProcess": {
#     "name": "CTCLabelDecode"  
#         },
#         "Metric": {
#             "name": "RecMetric"
#         }
#     }
    
#     config_file = Path(output_path) / 'config.yml'
#     with open(config_file, 'w') as f:
#         json.dump(config, f, indent=2)
    
#     print(f"Created configuration file at: {config_file}")

# def prepare_training():
#     """
#     Prepare all necessary files for PaddleOCR training
#     """
#     # Create output directory
#     output_path = Path('paddleocr_training')
#     output_path.mkdir(exist_ok=True)
    
#     # Create training lists
#     for split in ['train', 'val', 'test']:
#         create_training_list('dataset', output_path, split)
    
#     # Create dictionary
#     num_classes = create_dict_file('dataset', output_path)
    
#     # Create configuration
#     create_config(output_path, num_classes)
    
#     print("\nTraining preparation completed!")
#     print("\nTo start training, follow these steps:")
#     print("1. Install PaddlePaddle and PaddleOCR:")
#     print("   pip install paddlepaddle-gpu paddleocr")
#     print("\n2. Run training:")
#     print("   python PaddleOCR/tools/train.py -c paddleocr_training/config.yml")

In [4]:
!python PaddleOCR/tools/train.py -c paddleocr_training/config.yml

[2024/12/11 16:58:05] ppocr WARNING: You are using VisualDL, the VisualDL is deprecated and removed in ppocr!
[2024/12/11 16:58:05] ppocr INFO: Architecture : 
[2024/12/11 16:58:05] ppocr INFO:     Backbone : 
[2024/12/11 16:58:05] ppocr INFO:         layers : 34
[2024/12/11 16:58:05] ppocr INFO:         name : ResNet
[2024/12/11 16:58:05] ppocr INFO:     Head : 
[2024/12/11 16:58:05] ppocr INFO:         fc_decay : 0.0004
[2024/12/11 16:58:05] ppocr INFO:         name : CTCHead
[2024/12/11 16:58:05] ppocr INFO:     Neck : 
[2024/12/11 16:58:05] ppocr INFO:         encoder_type : rnn
[2024/12/11 16:58:05] ppocr INFO:         hidden_size : 256
[2024/12/11 16:58:05] ppocr INFO:         name : SequenceEncoder
[2024/12/11 16:58:05] ppocr INFO:     Transform : None
[2024/12/11 16:58:05] ppocr INFO:     algorithm : CRNN
[2024/12/11 16:58:05] ppocr INFO:     model_type : rec
[2024/12/11 16:58:05] ppocr INFO: Eval : 
[2024/12/11 16:58:05] ppocr INFO:     dataset : 
[2024/12/11 16:58:05] ppocr I

Traceback (most recent call last):
  File "d:\Project\License_plate\PaddleOCR\tools\train.py", line 269, in <module>
    main(config, device, logger, vdl_writer, seed)
  File "d:\Project\License_plate\PaddleOCR\tools\train.py", line 55, in main
    train_dataloader = build_dataloader(config, "Train", device, logger, seed)
                       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\Project\License_plate\PaddleOCR\ppocr\data\__init__.py", line 107, in build_dataloader
    dataset = eval(module_name)(config, mode, logger, seed)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\Project\License_plate\PaddleOCR\ppocr\data\simple_dataset.py", line 46, in __init__
    self.do_shuffle = loader_config["shuffle"]
                      ~~~~~~~~~~~~~^^^^^^^^^^^
KeyError: 'shuffle'
